### TOPIC Identification of Mathematical word problems
- In a smaller dataset , I have manually categorized the word problems and used Keras deep learning to classifiy the questions 


### Load the updated AQuA Dataset with classes 

In [1]:
import pandas as pd
import numpy as np
import json 

In [2]:
#qb =  pd.read_json('AQuA-master/dev.json',lines=True)
qbc =  pd.read_csv('dev-class5.csv',index_col=False)
#qbc.reset_index()
#qbc = qbc.join(qbc['QOR'].apply(json.loads).apply(pd.Series))


In [3]:
qbc.head()

,Unnamed: 0,correct,options,question,rationale,qn,clusters
0,0,A,"['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10']",Three birds are flying at a fast rate of 900 k...,To calculate the equivalent of miles in a kilo...,three birds are flying at a fast rate of kilo...,2
1,1,D,"['A)100 m', 'B)150 m', 'C)200 m', 'D)250 m', '...",A ship is leaving a port. It takes 240 seconds...,Let the length of the ship be x metres and its...,a ship is leaving a port. it takes seconds to...,3
2,2,C,"['A)6', 'B)18', 'C)24', 'D)36', 'E)48']",A rectangular piece of cloth 2 feet wide was c...,"The question says, length of shorter piece is ...",a rectangular piece of cloth feet wide was cu...,3
3,3,B,"['A)(1,1)', 'B)(0,3)', 'C)(2,0)', 'D)(3,6)', '...","In the xy-coordinate plane, which of the follo...",For a point to satisfy the given equation for ...,"in the xy-coordinate plane, which of the follo...",3
4,4,C,"['A)7', 'B)9', 'C)13', 'D)27', 'E)45']",A travel company wants to charter a plane to t...,"Additional passengers (i.e., an integer, let's...",a travel company wants to charter a plane to t...,3


### Remove Named Entities 

In [4]:
import nltk
import string
def untokenize(tokens):
    return("".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip())


def extract_nonentities(tree):
    tokens = [leaf[0] for leaf in tree if type(leaf) != nltk.Tree]
    return(untokenize(tokens))

def ne_removal(text_list):
    token_list = [nltk.word_tokenize(text) for text in text_list]
    tagged = nltk.pos_tag_sents(token_list)
    chunked = nltk.ne_chunk_sents(tagged)
    non_entities = []
    for tree in chunked:
        non_entities.append(extract_nonentities(tree))
    return(non_entities)

In [5]:
# removal of named entities
qbc['qn'] = ne_removal(qbc['question'])
#qbc['qn'] = (qbc['question'])



### Pre process text  - convert to lower case , remove numeric , special characters

In [6]:
import re
qbc.qn = qbc.qn.apply(lambda x: x.lower())
qbc['qn'] = qbc['qn'].apply(lambda x: re.sub(r'\%', 'percent', x))
qbc['qn'] = qbc['qn'].apply(lambda x:re.sub("[^a-zA-Z]",' ',x))


### Split the question into word list for vectorization

In [7]:
#qbc['qn']=qbc['qn'].apply(lambda x: x.split())
qbc.head()

,Unnamed: 0,correct,options,question,rationale,qn,clusters
0,0,A,"['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10']",Three birds are flying at a fast rate of 900 k...,To calculate the equivalent of miles in a kilo...,three birds are flying at a fast rate of k...,2
1,1,D,"['A)100 m', 'B)150 m', 'C)200 m', 'D)250 m', '...",A ship is leaving a port. It takes 240 seconds...,Let the length of the ship be x metres and its...,a ship is leaving a port it takes seconds...,3
2,2,C,"['A)6', 'B)18', 'C)24', 'D)36', 'E)48']",A rectangular piece of cloth 2 feet wide was c...,"The question says, length of shorter piece is ...",a rectangular piece of cloth feet wide was c...,3
3,3,B,"['A)(1,1)', 'B)(0,3)', 'C)(2,0)', 'D)(3,6)', '...","In the xy-coordinate plane, which of the follo...",For a point to satisfy the given equation for ...,in the xy coordinate plane which of the follo...,3
4,4,C,"['A)7', 'B)9', 'C)13', 'D)27', 'E)45']",A travel company wants to charter a plane to t...,"Additional passengers (i.e., an integer, let's...",a travel company wants to charter a plane to t...,3


### Tokenize all the word vectors

In [8]:
AllText = qbc['qn'].values

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(AllText)
vocab_size = len(t.word_index) + 1
print(vocab_size)


Using TensorFlow backend.


1573


In [33]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.3

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout,Concatenate
from keras.models import Model

In [16]:
texts =  qbc['qn'].values
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


labels=to_categorical(qbc['clusters'])
#labels = to_categorical(labelenc.transform(qbc["Topic"].values))

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 1572 unique tokens.
Shape of data tensor: (254, 1000)
Shape of label tensor: (254, 5)


In [34]:

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]



### Load the GloVe Word embeddings 

In [18]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors from GloVe.' % len(embeddings_index))

Loaded 400000 word vectors from GloVe.


### Create a word embeddings for the question vocabulary using the GloVe embeddings

In [19]:
# create a weight matrix for words in training docs
vocab = []
embedding_matrix = np.zeros((vocab_size, 300))
err_count = 0
for word, i in t.word_index.items():
    vocab.append(word)
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else :
        print("Can't find Glove word vector for :{}".format(word))
        err_count +=1
vocab.append("-null")

Can't find Glove word vector for :thepercent
Can't find Glove word vector for :prosthodontist
Can't find Glove word vector for :yuvaraj
Can't find Glove word vector for :playerwho
Can't find Glove word vector for :whatpercent
Can't find Glove word vector for :huhulians
Can't find Glove word vector for :dbef
Can't find Glove word vector for :koolaid
Can't find Glove word vector for :irder


In [20]:
print(embedding_matrix.shape)

(1573, 300)


In [35]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)

l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)

l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(5, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - convolutional neural network")
model.summary()
#model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=10, batch_size=128)
model.fit(x_train, y_train,validation_data=(x_val, y_val),epochs=15, batch_size=10)

model fitting - convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1000, 300)         471900    
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 996, 128)          192128    
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 199, 128)          0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 39, 128)           0         
_________________________________________________________________
conv1d_21 (Conv1D)           (N

In [36]:
scores = model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 64.47%


In [37]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [38]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [39]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=128,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

In [40]:
#l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(5, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [42]:
print("Model - complex CNN")
model.summary()
model.fit(x_train, y_train,validation_data=(x_val, y_val), epochs=15, batch_size=10)

Model - complex CNN
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1000, 300)    471900      input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 998, 128)     115328      embedding_6[0][0]                
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 997, 128)     153728      embedding_6[0][0]                
_________________________________________________________________________________________

In [43]:
scores = model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 65.79%
